In [1]:
# Import the libraries
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
# Load the last 3 years of data for AMZN
ticker = 'AMZN'
stock = yf.Ticker(ticker)
hist_data = stock.history(period='3y')

prices = hist_data['Close']
prices

Date
2021-05-10 00:00:00-04:00    159.524506
2021-05-11 00:00:00-04:00    161.195496
2021-05-12 00:00:00-04:00    157.597000
2021-05-13 00:00:00-04:00    158.073502
2021-05-14 00:00:00-04:00    161.145004
                                ...    
2024-05-02 00:00:00-04:00    184.720001
2024-05-03 00:00:00-04:00    186.210007
2024-05-06 00:00:00-04:00    188.699997
2024-05-07 00:00:00-04:00    188.759995
2024-05-08 00:00:00-04:00    188.000000
Name: Close, Length: 755, dtype: float64

In [3]:
# Fix the parameters for the risk analysis
alpha = 0.99
num_shares = 5000
on_date = '2024-05-08'
share_price = prices[on_date]

In [4]:
# Compute the return rates
return_rates = prices.pct_change()
return_rates

Date
2021-05-10 00:00:00-04:00         NaN
2021-05-11 00:00:00-04:00    0.010475
2021-05-12 00:00:00-04:00   -0.022324
2021-05-13 00:00:00-04:00    0.003024
2021-05-14 00:00:00-04:00    0.019431
                               ...   
2024-05-02 00:00:00-04:00    0.031955
2024-05-03 00:00:00-04:00    0.008066
2024-05-06 00:00:00-04:00    0.013372
2024-05-07 00:00:00-04:00    0.000318
2024-05-08 00:00:00-04:00   -0.004026
Name: Close, Length: 755, dtype: float64

In [5]:
# Delete the NaN
return_rates = return_rates.dropna()
return_rates

Date
2021-05-11 00:00:00-04:00    0.010475
2021-05-12 00:00:00-04:00   -0.022324
2021-05-13 00:00:00-04:00    0.003024
2021-05-14 00:00:00-04:00    0.019431
2021-05-17 00:00:00-04:00    0.014735
                               ...   
2024-05-02 00:00:00-04:00    0.031955
2024-05-03 00:00:00-04:00    0.008066
2024-05-06 00:00:00-04:00    0.013372
2024-05-07 00:00:00-04:00    0.000318
2024-05-08 00:00:00-04:00   -0.004026
Name: Close, Length: 754, dtype: float64

In [6]:
# Get the quantile
quantile = np.quantile(return_rates, q=1-alpha,
                      method='inverted_cdf')
quantile

-0.06804249715805744

In [7]:
# Compute the mean of return rates
mean_return_rate = return_rates.mean()
mean_return_rate

0.0004960944005615992

In [8]:
# Compute the non-parametric VaR
VaR_np = num_shares * share_price * (mean_return_rate - quantile)
print('The n.p. VaR is:', VaR_np)

The n.p. VaR is: 64426.27606510189


In [9]:
# VaR PARAMETRIC VERSION
from scipy.stats import norm

# Normal return assumption
# Use the z-value, mean and variance to calculate VaR
z_value = norm.ppf(1 - alpha)

In [10]:
# Mean and standard deviation of return rates
mean_return_rate = return_rates.mean()
std_return_rate = return_rates.std()

In [11]:
mean_return_rate

0.0004960944005615992

In [12]:
std_return_rate

0.023608084948699298

In [13]:
# Compute the (1-alpha)-th quantile
quantile = mean_return_rate + std_return_rate * z_value
quantile

-0.05442452383002058

In [14]:
# Compute the VaR under normality
VaR_normal = num_shares * share_price * (mean_return_rate - quantile)

In [15]:
print('The normal VaR is:', VaR_normal)

The normal VaR is: 51625.38113674725


In [16]:
# Total investment
num_shares * share_price

940000.0

In [17]:
# Compute the absolute VaR
VaR_normal_abs = num_shares * share_price * (0 - quantile)
print('The absoulte VaR is:', VaR_normal_abs)

The absoulte VaR is: 51159.05240021935
